In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import math
from numpy.linalg import inv
import numpy.matlib
from scipy.stats.distributions import chi2
import warnings
warnings.filterwarnings('ignore')
import scipy.linalg as la
import sys

from plot_histo import plot_histo

plt.rcParams['figure.figsize'] = 8, 8
plt.rcParams['lines.linewidth'] = 1

In [ ]:
# genlsq = Generalized Least Squares
# Template script for the iterative quasi-Newton method for a 4-parameter
# inversion for epicenter (xs, ys), origin time (ts), and velocity (V).
# The algorithm employs generalized least squares, where by both data
# covariances and model covariances are used.
#
# Background reading: Tarantola book (2005), Ch. 3 and Appendix 6.22
#
# calls forward_epicenter.ipynb, plot_histo()
#
# contributors: Carl Tape, Nealey Sims*

In [ ]:
# USER INPUT
nsamples = 1000;
irandom_initial_model = 0;      # 0(fixed), 1(random)
irandom_target_model = 0;       # 0(fixed), 1(random)
idata_errors = 2;               # 0(none),  1(random), 2(fixed)
ifig = 1;                       # 0,1

inormalization = 1;
stnsamples = str(nsamples) + ' samples'
stlabS = ('Sd(m^k)','Sm(m^k)','S(m^k) = Sd + Sm')

In [ ]:
# FORWARD PROBLEM: EXPLORE THIS SEPARATE NOTEBOOK   
%run ./forward_epicenter.ipynb

In [ ]:
# predictions for prior and initial models (these are not used)
dprior   = d(mprior)
dinitial = d(minitial);

if ifig==1:
    # plot different histograms of properties of the prior model covariance samples
    fig=plt.figure(figsize=(9,9))
    for kk in range(nparm):
        sigma = sigma_prior[kk]
        #edges = [-4*sigma: sigma/2 : 4*sigma]
        edges=np.arange(-4*sigma,4*sigma,sigma/2)
        etemp = cov_samples_m[kk,:]
        plt.subplot(2,2,kk+1)
        plt.grid()
        plot_histo(etemp,edges,2);
        #plt.bar(etemp)
        plt.ylim([0, 0.4])
        plt.title('mprior samples: Model parameter '+str(kk+1)+' ('+str(mlabs[kk])+')\n'+'mean = %.5f; std = %.5f' % (np.mean(etemp),np.std(etemp)))
        plt.tight_layout()
    # plot different histograms of properties of the data covariance samples
    fig2 = plt.figure(figsize=(9,11))
    for ii in range(ndata):
        sigma = sigma_obs[ii];
        edges=np.arange(-4*sigma,4*sigma,sigma/2)
        etemp = cov_samples_d[ii,:]
        plt.subplot(4,3,ii+1); 
        plot_histo(etemp,edges); 
        plt.ylim([0, 0.4]);
        plt.title('Data index ' + str(ii+1) +'\n'+'mean = %.5f; std = %.5f' % (np.mean(etemp),np.std(etemp))) 
        plt.tight_layout()
    fig3 = plt.figure(figsize=(10,10))
    plt.plot(dobs_samples,'.-');
    p1 = plt.plot(dprior,'bo-',linewidth=2,markersize=10,markerfacecolor='b',markeredgecolor='w')
    p2 = plt.plot(dinitial,'ko-',linewidth=2,markersize=10,markerfacecolor='k',markeredgecolor='w');
    p3 = plt.plot(dtarget,'ro--',linewidth=2,markersize=10,markerfacecolor='r',markeredgecolor='w');
    p4 = plt.plot(dobs,'ro-',linewidth=2,markersize=10,markerfacecolor='r',markeredgecolor='w');
    plt.legend([p1[0], p2[0], p3[0], p4[0]],['g(mprior)','g(minitial)','g(mtarget)','g(mtarget) + errors',
        'location','northwest'])
    #title(' BLACK = d(mprior);  RED DASHED = d(mtarget);  RED = d(mtarget) + errors');
    plt.xlim([0.5, ndata+0.5]); #set(gca,'xtick',[1:ndata]);
    plt.xlabel('Data index'); plt.ylabel('Prediction value, g(m)');

In [ ]:
# MISFIT FUNCTION: least squares, Tarantola (2005), Eq. 6.251
# This calls the function d(m) to compute the predictions.

# data misfit
def Sd(m,dobs,icobs):
    sd=((np.dot(0.5,(d(m)-dobs).T))@icobs)@(d(m)-dobs)
    return sd
# model misfit (related to regularization)
def Sm(m,mprior,icprior):
    sm=((np.dot(0.5,(m-mprior).T))@icprior)@(m-mprior)
    return sm
# total misfit
def S(m,dobs,mprior,icobs,icprior):
    s=Sd(m,dobs,icobs) + Sm(m,mprior,icprior)
    return s

# initial model
#mnew = mprior;     # prior model
#mnew = mtarget;    # target model
mnew = minitial;
Sd_0 = Sd(mnew,dobs,icobs);
Sm_0 = Sm(mnew,mprior,icprior);
S_0  = S(mnew,dobs,mprior,icobs,icprior);
stS0 = ' S(m0) = %.3f = %.3f(D) + %.3f(M)'% (S_0,Sd_0,Sm_0)
print(stS0);

In [ ]:
niter = 0
#niter = input(' Select the number of iterations (< 10) or 0 to exit: ');
#niter = int(niter)
if int(niter) == 0:
    sys.exit('no iterations requested: exiting')

In [ ]:
# initialize arrays
iter_vec = np.transpose(np.arange(0,niter+1))
Sd_vec = np.zeros((niter+1,1));
Sm_vec = np.zeros((niter+1,1));
S_vec = np.zeros((niter+1,1));

# misfit for initial model
Sd_vec[0] = Sd_0;
Sm_vec[0] = Sm_0;
S_vec[0]  = S_0;

for nn in range(1,niter+1):
    #///////////////////////////////
    # CODE HERE for quasi-Newton algorithm

    
    
    

    # misfit function for new model
    # note: book-keeping only -- not used within the algorithm above
    #Sd_vec[nn] = 
    #Sm_vec[nn] = 
    #S_vec[nn] = 

    #///////////////////////////////

# misfit function values
print('summary of misfit function:');
print('%8s%16s%16s%16s'% ('iter','Sd','Sm','S = Sm + Sd'))
for nn in range(niter):
    print('%8i%16.10f%16.10f%16.10f' % (iter_vec[nn],Sd_vec[nn],Sm_vec[nn],S_vec[nn]))

In [ ]:
if ifig==1:
    # plot convergence curve
    ylims = [10**-2, 10**2];
    stit = str(niter) +' iterations'
    plt.plot(iter_vec, np.log10(Sd_vec),'r.-',iter_vec, np.log10(Sm_vec),'b.-',iter_vec, np.log10(S_vec),'k.-',
        linewidth=2,markersize=20)
    plt.legend(stlabS); plt.xlim([-0.5, niter+0.5]); plt.ylim(np.log10(ylims))
    plt.locator_params(axis="x", integer=True, tight=True)
    plt.xlabel('k, iteration'); plt.ylabel(' log10[ S(m^k) ], misfit function'); plt.title(stit)

In [ ]:
def corrcov(cov):
    # convert correlation matrix from covariance matrix
    v = np.sqrt(np.diag(cov))
    outer_v = np.outer(v, v)
    corr = cov / outer_v
    corr[cov == 0] = 0
    return corr

In [ ]:
#///////////////////////////////
# COMPUTE THE FOLLOWING
# mpost       posterior model ("final" model)
# dpost       predictions for mpost
# Gpost       partial derivatives matrix at mpost
# cpost0      posterior covariance matrix (use icobs0 and icprior0)
# sigma_post  variances of the posterior covariance matrix
# rho_post    posterior correlation matrix (hint: see Tarantola, Section 3.3)
#
# CODE HERE




#///////////////////////////////

In [ ]:
# a priori model correlations (for comparison)
rho_prior = corrcov(cprior0)
print('cpost0 \n', cpost0, '\n', 'rho_post \n', rho_post)

# posterior data covariance matrix (e.g., Tarantola Eq. 3.44)
cpost0_d = Gpost@cpost0@Gpost.T
cpost0_d = (cpost0_d + cpost0_d.T)/2       # force to be symmetric
sigma_post_d = np.sqrt(np.diag(cpost0_d))  # IGNORING OFF-DIAGONAL ELEMENTS
rho_post_d = corrcov(cpost0_d)             # posterior correlation matrix
rho_prior_d = corrcov(cobs0)               # prior, for comparison

print('model summary (%i iterations):'% (niter))
print('%16s%16s%16s%16s' % ('prior', 'initial','posterior', 'target'))
for ii in range(nparm):
        print('%16s%16s%16s%16s'%(str(mprior[ii]),str(minitial[ii]),str(mpost[ii]), str(mtarget[ii])))
print('data summary (%i observations):' % (ndata))
print('%16s%16s%16s%16s%16s'% ('prior', 'initial','posterior', 'target', 'actual'))
for ii in range(ndata):
        print('%16s%16s%16s%16s%16s'%(str(dprior[ii]),str(dinitial[ii]),str(dpost[ii]), 
                                      str(dtarget[ii]), str(dobs[ii])))
# Cholesky decomposition to obtain the square-root of cpost0
# NOTE: for large problems, this is not possible due to poor
#       conditioning of cpost0 or the inability to compute cpost0
Lpost = np.linalg.cholesky(cpost0);

# samples of the posterior distribution
mpost_samples = np.zeros((nparm,nsamples))
mcov_samples = np.zeros((nparm,nsamples))
for xx in range(nsamples): 
    randn_vecs_m[:,xx] = np.random.randn(nparm)
mcov_samples  = Lpost @ randn_vecs_m;
mpost_samples = np.matlib.repmat(mpost,1,nsamples) + mcov_samples;

# compare the standard deviation with sigma_post

std_samples = np.std(mpost_samples.T, axis=0);

# compare posterior model distribution with prior
# note: format statement allows for vectors (like sigma_prior)
print('  ');
print(' Compare model uncertainties : ');
print('             model parameter : %13s%13s%13s%13s' %(mlabs[0],mlabs[1],mlabs[2],mlabs[3]))
print('                       units : %13s%13s%13s%13s' %(ulabs[0],ulabs[1],ulabs[2],ulabs[3]));
print('                 sigma_prior = %13.5s%13.5s%13.5s%13.5s'%(sigma_prior[0],sigma_prior[1],
                                                                 sigma_prior[2],sigma_prior[3]))
print('                  sigma_post = %13.5s%13.5s%13.5s%13.5s'%(sigma_post[0],sigma_post[1],
                                                                 sigma_post[2],sigma_post[3]));
print('   std(%6.0f mpost_samples) = %13.5s%13.5s%13.5s%13.5s' % (nsamples, std_samples[0],std_samples[1],
                                                                  std_samples[2],std_samples[3]));
print('    sigma_post / sigma_prior = %13.5s%13.5s%13.5s%13.5s' % (np.divide(sigma_post[0],sigma_prior[0]),
                                                                  np.divide(sigma_post[1],sigma_prior[1]),
                                                                  np.divide(sigma_post[2],sigma_prior[2]),
                                                                  np.divide(sigma_post[3],sigma_prior[3])));
print('1 - sigma_post / sigma_prior = %13.5s%13.5s%13.5s%13.5s' % (1 - np.divide(sigma_post[0],sigma_prior[0]),
                                                                  1 - np.divide(sigma_post[1],sigma_prior[1]),
                                                                  1 - np.divide(sigma_post[2],sigma_prior[2]),
                                                                  1 - np.divide(sigma_post[3],sigma_prior[3])))
print('  ')              

# compute the predictions associated with the posterior samples,
# then compare std_d_samples with sigma_post_d
d_samples = np.zeros((ndata,nsamples))
for xx in range(nsamples):
    ms = mpost_samples[:,xx]
    d_samples[:,xx] = d(ms)

covd_samples = np.cov(d_samples)
rhod_samples = corrcov(covd_samples)
std_d_samples = np.sqrt(np.diag(covd_samples))
#std_d_samples = std(d_samples');

print('  ');
print(' Compare data uncertainties : ')
print('%16s %10s %10s %10s'%('prior','post','samples','post/prior'))
call = [sigma_obs, sigma_post_d, std_d_samples, np.divide(sigma_post_d,sigma_obs)];
for ii in range(ndata):
    print('%6i%10.4f %10.4f %10.4f %10.4f'%(ii+1,call[0][ii],call[1][ii],call[2][ii],call[3][ii]))
print('  ');

In [ ]:
def plot_covsamples(msamples,rho,tlab,sym1,msamples2,rho2,tlab2,sym2,mlabs):
    # generates plots for samples of covariance matrix
    #
    # INPUT
    #    msamples   n x p matrix of vector samples
    #    rho        n x n 'analytical' correlation matrix
    #    tlab       label for plot
    #    sym1       plotting symbol
    #    msamples2  optional: 2nd set of samples ([] for none)
    #    rho2       optional: 2nd 'analytical' correlation matrix ([] for none)
    #    tlab2      optional: label for plot
    #    sym2       optional: plotting symbol
    #    mlabs      optional: labels for each variable ([] for default)
    #
    # EXAMPLE: 
    #    plot_covsamples(mpost_samples,rho_post,'mpost','b.',[],[],[],[],mlabs);
    #
    # NOTE: We could alternatively estimate the covariance matrix
    # (and correlation matrix) directly from the input samples.
    #
    # Carl Tape, 2012-01-01

    NMAX = 6;   # max number to make into scatterplot
    
    n = np.shape(msamples)[0]
    p = np.shape(msamples)[1]
    print('plot_covsamples: n = %i, p = %i'%(n,p))
    
    if len(mlabs)==0:
        #mlabs = strtrim(cellstr(num2str([1:n]')));
        mlabs = []#np.matlib.repmat(str(''),n,1);
        for ii in range(n):
            mlabs.append( 'i'+str(ii+1))        
    # whether to plot a second set of samples
    if len(msamples2)!=0 and len(rho2)!=0:
        isecond = 1; 
    else:
        isecond = 0
    
    # kk=1: correlation matrices from Cpost
    # kk=2: correlation matrices based on input SAMPLES
    fig=plt.figure(figsize=(8,8));
    
    nr=1+isecond; nc=2;
    for kk in [0,1]:
        if kk==0:
            F1 = rho; 
            if isecond==1:
                F2 = rho2
            stag = ''
        else:
            F1 = np.corrcoef(msamples);
            if isecond==2:
                F2 = np.corrcoef(msamples2)
            stag = 'sample'
        
        # first matrix plot
        pind = kk+1+isecond*(kk);
        ax = fig.add_subplot(nr,nc,pind)
        p1=ax.imshow(F1,cmap='jet', vmin=-1,vmax=1);
        ax.xaxis.tick_top()
        if isecond==0:
            fig.colorbar(p1,shrink=0.3, aspect=10,ticks=[-1, -0.5, 0, 0.5, 1])
        else:
            fig.colorbar(p1,shrink=0.7, aspect=10,ticks=[-1, -0.5, 0, 0.5, 1])
        if len(mlabs)==4:
            plt.xticks([0,1,2,3],mlabs)
            plt.yticks([0,1,2,3],mlabs)
        else:
            plt.xticks(np.arange(0,n,1),mlabs)
            plt.yticks(np.arange(0,n,1),mlabs)
        plt.title(str(stag)+ ' correlation matrix for '+ str(tlab));
        
        # second matrix plot (OPTIONAL)
        if isecond==1:
            ax2 = fig.add_subplot(nr,nc,pind+1)
            p2=ax2.imshow(F2, cmap='jet', vmin=-1,vmax=1); #caxis([-1 1]), colorbar
            ax2.xaxis.tick_top()
            if len(mlabs)==4:
                plt.xticks([0,1,2,3],mlabs)
                plt.yticks([0,1,2,3],mlabs)
            else:
                plt.xticks(np.arange(0,n,1),mlabs)
                plt.yticks(np.arange(0,n,1),mlabs)
            plt.title(str(stag)+ ' correlation matrix for '+ str(tlab2));
            fig.colorbar(p2,shrink=0.7, aspect=10,ticks=[-1, -0.5, 0, 0.5, 1])
    plt.tight_layout()
    
    # scatterplots
    if n > NMAX:
        print('n = %i is > %i, so no scatterplots made'% (n,NMAX))
    else:
        fig=plt.figure(figsize=(8,8)); 
        nr=n-1; nc=n-1;
        for ii in range(n-1):
            jj=ii+1
            while jj<n: 
                px = np.array([msamples[ii,:]])
                py = np.array([msamples[jj,:]])
                iplot = nc*(ii) + jj;
                #disp([ii jj iplot]);
                ax = fig.add_subplot(nr,nc,iplot)
                ax.plot(px,py,sym1,markersize=2);
                ax.xaxis.tick_bottom()
                plt.xlabel(mlabs[ii]); plt.ylabel(mlabs[jj]);
                st1 = 'corr(%s) = %.2f (%.2f)'% (tlab,np.corrcoef(px,py,ddof=0)[0,1],rho[ii,jj]);
                if isecond==1:
                    px = msamples2[ii,:]
                    py = msamples2[jj,:]
                    plt.plot(px,py,sym2,markersize=2);
                    st2 = 'corr(%s) = %.2f (%.2f)'% (tlab2,np.corrcoef(px,py,ddof=0)[0,1],rho2[ii,jj])
                    plt.title(str(st1)+'\n'+ str(st2));
                else:
                    plt.title(str(st1));
                jj+=1
    plt.tight_layout()
    plt.show()

In [ ]:
if ifig==1:
    # display distributions for each model parameter (nparm ROWS of cov_samples_m)
    fig=plt.figure(figsize=(8,8));
    nr=2; nc=2;
    for kk in range(nparm):
        sigma = sigma_post[kk];
        edges=np.arange(-4*sigma,4*sigma,sigma/2)
        etemp = mcov_samples[kk,:]
        plt.subplot(nr,nc,kk+1); 
        plot_histo(etemp,edges); 
        plt.ylim([0, 0.4]); 
        plt.grid
        stl1 = 'mpost samples'
        stl2 = 'Model parameter ' +str(kk+1) +' (' +str(mlabs[kk]) +')'
        stl3 = 'mean = %.5f; std = %.5f' % (np.mean(etemp),np.std(etemp))

        if kk==0: 
            plt.title(str(stl1) + ': ' + str(stl2) + '\n'+str(stl3))
        else: 
            plt.title(str(stl2) + '\n'+str(stl3))
    plt.tight_layout()
    # correlation matrices and scatterplots
    #plot_covsamples(mprior_samples,rho_prior,'mprior','b.',[],[],[],[],mlabs);
    plot_covsamples(mpost_samples,rho_post,'mpost','r.',[],[],[],[],mlabs);
    plot_covsamples(mprior_samples,rho_prior,'mprior','b.',mpost_samples,rho_post,'mpost','r.',mlabs);

    # 'physical view' of estimated posterior data uncertainties
    # note: plot either sigma_post_d (from Cpost_d) or std_d_samples (from d(Cpost_samples))
    plt.figure(figsize=(8,8))
    plt.subplot(aspect=1)
    plt.plot(mpost_samples[0,:],mpost_samples[1,:],'c.');
    plt.plot(mpost[0],mpost[1],'o',markersize=10,markerfacecolor='c',markeredgecolor='w');
    #plot(mprior(1),mprior(2),'o','markersize',10,'markerfacecolor','b','markeredgecolor','w');
    #scatter(xrec,yrec,16^2,sigma_post_d,'filled','V'); title('estimated uncertainties for posterior predictions');

    plt.scatter(xrec,yrec,16**2,std_d_samples,marker='v',edgecolors='k', cmap='hot'); 
    #plt.scatter(xrec,yrec,16**2,'k',marker='v', cmap='hot');
    plt.xlim([0,100]);plt.ylim([0,100])
    #set(gca,'xtick',[0:20:100],'ytick',[0:20:100]);
    plt.xlabel('X distance (km)'); plt.ylabel('Y distance (km)');
    plt.colorbar(shrink=0.8);
    plt.title('uncertainties for posterior predictions, computed from samples');
    plt.show()
    # plot predictions for samples of the posterior
    plot_covsamples(d_samples,rho_post_d,'dpost','k.',[],[],[],[],[]);

    #plt.subplot(aspect=1)
    # note: opts is set in forward_epicenter
    plot_epicenters(mprior_samples,mprior,minitial,mtarget,opts,mpost);
    # plot the cpost0 samples and re-plot the two markers
    plt.plot(mpost_samples[0,:],mpost_samples[1,:],'c.')
    plt.plot(mpost[0],mpost[1],'o',markersize=10,markerfacecolor='c',markeredgecolor='w')
    plt.plot(mtarget[0],mtarget[1],'o',markersize=10,markerfacecolor='r',markeredgecolor='w')
    plt.plot(minitial[0],minitial[1],'o',markersize=10,markerfacecolor='k',markeredgecolor='w')
    #plt.xlim([0,100]);plt.ylim([0,100]);plt.axis('equal')
    plt.title('samples of prior (blue) and posterior (cyan)');
    plt.tight_layout()